## Azure ML Service to operationalize the TF model

Now, we are going to use the Azure ML Service Python SDK to create an image using our custom trained model and deploy this as a web service.

Install the AzureML SDK v1.0.8 and follow the steps below.

To verify the version of the SDK, import the module and query the version information -

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.0.8


Creating the workspace require authentication and uses the subscription ID for this process. Provide a unique name for workspace, subscription if, resource group name (which will be created if not existing) and location if needed. This will initiate interactive authentication. Please use the credential provided in the lab UI to connect to the right subscription - 

In [ ]:
# Create a new workspace if needed, takes about 2 minutes
from azureml.core import Workspace
ws = Workspace.create(name='rlab314ws',
                      subscription_id='subscription-id', 
                      resource_group='rlab314wsrg',
                      create_resource_group=True,
                      location='eastus2' # Or other supported Azure region   
                     )

You can save the workspace informtion to reuse later -

In [5]:
# Save the config 
ws.write_config()

Wrote the config file config.json to: /home/labadmin/aml_config/config.json


If you have saved the workspace information, use the following to reload instead of creating a new one from the scratch -

In [ ]:
# Connect to existing workspace
from azureml.core import Workspace
ws=Workspace.from_config()

## Declaring depencies

Use the following code to create the environment for your model to run and evaluate the input. Use the following code to create an environment file which will be used to inject the required module when the image is built -

In [7]:
# Declare dependencies
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("numpy")
myenv.add_conda_package("pillow")
myenv.add_conda_package("tensorflow")
myenv.add_conda_package("matplotlib")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
#with open("myenv.yml","r") as f:
#    print(f.read())

Next step is to register your model with the workspace. You can either point the register function to your model file directly or if you need additional artefacts for your model to function correctly, you can store them in a directory and point this code to that directory instead -

In [11]:
# Register a trained model
from azureml.core.model import Model

model = Model.register(model_path = "tfeval",
                       model_name = "icondetection",
                       description = "icon detection model using TF",
                       workspace = ws)

print(model.version)

Registering model icondetection


We will now create a configuration for the image. This step requires a scoring script which process your inputs once deployed as a web service. Also needed is the information on the runtime and the environment file created earlier -

In [9]:
# Create and register an image
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                 runtime = "python",
                                                 conda_file = "myenv.yml",
                                                 description = "icon detection model image using TF",
                                                 )

Create this image using the code below. This will build the image using the information we have declared above such as the scoring script, runtime and environment file and register it the Azure Container Registry that gets created in the workspace creation steps above -

In [10]:
# Register the image from the image configuration in ACR - takes 3-5 mins
image = ContainerImage.create(name = "icondetection", 
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )

image.wait_for_creation(show_output=True)

Creating image
Running.........................................................
SucceededImage creation operation finished for image icondetection:1, operation "Succeeded"


If the image creation fails for any reason, you get your hands on the build log to investigate the errors -

In [ ]:
# Get the logs if needed
#print(image.image_build_log_uri)

It's time to push create a web service. Using the Azure ML SDK, you can used this image to to creat web service running on Azure Container Instance, AKS, IOT Edge or even, FPGA. In this example, we are using ACI to create a configuration of the container group and use the image that we created earlier to create a web service -  

In [12]:
# Deploy to ACI - takes 3-4 mins
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 2, 
                                               tags = {"data": "icon detection web service", "type": "icon detection"}, 
                                               description = 'TF Object Detection model for icon detection')

from azureml.core.webservice import Webservice

service_name = 'aci-id-1'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

Creating service
Running....................................
SucceededACI service creation operation finished, operation "Succeeded"


You can get the service state and if needed the deployment logs to investigate if there were any error in the process -


In [13]:
print(service.state)
#service.get_logs()

Healthy


If the deployment was succesfull, you can use the scoring_uri() to get the scoring URL which you can call from your applications -

In [14]:
# Get the scorig URI
service.scoring_uri

'http://52.151.224.120:80/score'

Following code can be used to call this endpoint with the required parameters to get the predictions from the model -

In [15]:
# Call the endpoint
import requests
import json
import numpy as np

input_data = "{\"data\": [\"some data\"]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

POST to url http://52.151.224.120:80/score
prediction: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>

